### unix command aql to connect to aerospike
```shell
Usage: 
$aql OPTIONS   
OPTIONS:
    -h <host>
        The hostname to the server connect to. Default: 127.0.0.1
    -p <port>
        The port number of the server to connect to. Default: 3000
    -U <user name>
        User name used to authenticate with cluster. Default: none
    -P[<password>]
        Password used to authenticate with cluster. Default: none
        User will be prompted on command line if -P specified and no password is given.
    -c <command>
        Execute the specified command.
    -f <filepath>
        Execute the commands in the specified file.
    -v
        Enable verbose output. Default: disabled
    -e
        Enable echoing of commands. Default: disabled
    -M
        Disable result display. Default: enabled
    -q
        Scan queue size. Default: 3
    -T <milliseconds>
        Set the timeout (ms) for commands. Default: 1000
    -l
        Enable durable delete. Default: disabled
    -K
        Enable sending keys to the server. Default: disabled
    -z <thread_pool_size>
        Set the number of client threads used to talk to the server. Default: 16
    -S
        Enable SLAP mode for queries. Default: disabled
    -t <n>
        Number of batch threads for SLAP mode. Default: 10
    -i <n>
        Number of iterations per thread for SLAP mode. Default: 10
    -o (json | table)
        Set the output mode. Default: table
    -n
        Disable outputting types for values (e.g., GeoJSON, JSON) to distinguish them from generic strings
    -F <file path>
        Set output file path. Default: /dev/null
    -u <path>
        Path to User managed UDF modules.
        Default: /opt/aerospike/usr/udf/lua
    -s <path>
        Path to the System managed UDF modules.
        Default: /opt/aerospike/sys/udf/lua
    -d
        Run in debug mode.
    -O, --options
        Print command-line options message.
    -V, --version
        Print AQL version information.
    -E, --help
        Print command-line options message plus documentation for AQL commands.
```

Examples: to connect aerospike from aws:    

```shell
$aql -h 10.0.0.180
```   

### aql commands
##### DDL
        CREATE INDEX <index> ON <ns>[.<set>] (<bin>) NUMERIC|STRING|GEO2DSPHERE
        CREATE LIST/MAPKEYS/MAPVALUES INDEX <index> ON <ns>[.<set>] (<bin>) NUMERIC|STRING|GEO2DSPHERE
        DROP INDEX <ns>[.<set>] <index>
        REPAIR INDEX <index> ON <ns>[.<set>]
            <ns> is the namespace for the index.
            <set> is the set name for the index.
            <index> is the name of the index.
Examples:

    CREATE INDEX gender_idx ON dmp.users (gender) STRING
    CREATE INDEX yob_idx ON dmp.users (yob) NUMERIC
    DROP INDEX dmp.users yob_idx
    REPAIR INDEX yob_idx ON dmp.users

##### DML
        INSERT INTO <ns>[.<set>] (PK, <bins>) VALUES (<key>, <values>)
        DELETE FROM <ns>[.<set>] WHERE PK = <key>

            <ns> is the namespace for the record.
            <set> is the set name for the record.
            <key> is the record's primary key.
            <key> is the record's primary key.
            <bins> is a comma-separated list of bin names.
            <values> is comma-separated list of bin values, which may include type cast expressions. Set to NULL (case insensitive & w/o quotes) to delete the bin.

          Type Cast Expression Formats:

              CAST(<Value> AS <TypeName>)
              <TypeName>(<Value>)

          Supported AQL Types:

                Bin Value Type                    Equivalent Type Name(s)
             ===============================================================
              Integer                           DECIMAL, INT, NUMERIC
              Floating Point                    FLOAT, REAL
              Aerospike CDT (List, Map, etc.)   JSON
              Aerospike List                    LIST
              Aerospike Map                     MAP
              GeoJSON                           GEOJSON
              String                            CHAR, STRING, TEXT, VARCHAR
             ===============================================================

          [Note:  Type names and keywords are case insensitive.]

Examples:

    INSERT INTO test.demo (PK, foo, bar) VALUES ('key1', 123, 'abc')
    INSERT INTO test.demo (PK, foo, bar) VALUES ('key1', CAST('123' AS INT), JSON('{"a": 1.2, "b": [1, 2, 3]}'))
    INSERT INTO test.demo (PK, foo, bar) VALUES ('key1', LIST('[1, 2, 3]'), MAP('{"a": 1, "b": 2}'))
    INSERT INTO test.demo (PK, gj) VALUES ('key1', GEOJSON('{"type": "Point", "coordinates": [123.4, -456.7]}'))
    DELETE FROM test.demo WHERE PK = 'key1'

##### QUERY
        SELECT <bins> FROM <ns>[.<set>]
        SELECT <bins> FROM <ns>[.<set>] WHERE <bin> = <value>
        SELECT <bins> FROM <ns>[.<set>] WHERE <bin> BETWEEN <lower> AND <upper>
        SELECT <bins> FROM <ns>[.<set>] WHERE PK = <key>
        SELECT <bins> FROM <ns>[.<set>] IN <indextype> WHERE <bin> = <value>
        SELECT <bins> FROM <ns>[.<set>] IN <indextype> WHERE <bin> BETWEEN <lower> AND <upper>
        SELECT <bins> FROM <ns>[.<set>] IN <indextype> WHERE <bin> CONTAINS <GeoJSONPoint>
        SELECT <bins> FROM <ns>[.<set>] IN <indextype> WHERE <bin> WITHIN <GeoJSONPolygon>

            <ns> is the namespace for the records to be queried.
            <set> is the set name for the record to be queried.
            <key> is the record's primary key.
            <bin> is the name of a bin.
            <value> is the value of a bin.
            <indextype> is the type of a index user wants to query. (LIST/MAPKEYS/MAPVALUES)
            <bins> can be either a wildcard (*) or a comma-separated list of bin names.
            <lower> is the lower bound for a numeric range query.
            <upper> is the lower bound for a numeric range query.

Examples:

    SELECT * FROM test.demo
    SELECT * FROM test.demo WHERE PK = 'key1'
    SELECT foo, bar FROM test.demo WHERE PK = 'key1'
    SELECT foo, bar FROM test.demo WHERE foo = 123
    SELECT foo, bar FROM test.demo WHERE foo BETWEEN 0 AND 999
    SELECT * FROM test.demo WHERE gj CONTAINS CAST('{"type": "Point", "coordinates": [0.0, 0.0]}' AS GEOJSON)

##### MANAGE UDFS
        REGISTER MODULE '<filepath>'
        SHOW MODULES
        REMOVE MODULE <filename>
        DESC MODULE <filename>

            <filepath> is file path to the UDF module(in single quotes).
            <filename> is file name of the UDF module.

Examples:

    REGISTER MODULE '~/test.lua'
    SHOW MODULES
    DESC MODULE test.lua
    REMOVE MODULE test.lua

##### INVOKING UDFS
        EXECUTE <module>.<function>(<args>) ON <ns>[.<set>]
        EXECUTE <module>.<function>(<args>) ON <ns>[.<set>] WHERE PK = <key>
        AGGREGATE <module>.<function>(<args>) ON <ns>[.<set>] WHERE <bin> = <value>
        AGGREGATE <module>.<function>(<args>) ON <ns>[.<set>] WHERE <bin> BETWEEN <lower> AND <upper>

            <module> is UDF module containing the function to invoke.
            <function> is UDF to invoke.
            <args> is a comma-separated list of argument values for the UDF.
            <ns> is the namespace for the records to be queried.
            <set> is the set name for the record to be queried.
            <key> is the record's primary key.
            <bin> is the name of a bin.
            <value> is the value of a bin.
            <lower> is the lower bound for a numeric range query.
            <upper> is the lower bound for a numeric range query.

Examples:

    EXECUTE myudfs.udf1(2) ON test.demo
    EXECUTE myudfs.udf1(2) ON test.demo WHERE PK = 'key1'
    AGGREGATE myudfs.udf2(2) ON test.demo WHERE foo = 123
    AGGREGATE myudfs.udf2(2) ON test.demo WHERE foo BETWEEN 0 AND 999

##### INFO
        SHOW NAMESPACES | SETS | BINS | INDEXES
        SHOW SCANS | QUERIES
        STAT NAMESPACE <ns> | INDEX <ns> <indexname>
        STAT SYSTEM
        ASINFO <ASInfoCommand>

##### JOB MANAGEMENT
        KILL_QUERY <transaction_id>
        KILL_SCAN <scan_id>

##### USER ADMINISTRATION
        CREATE USER <user> PASSWORD <password> ROLE[S] <role1>,<role2>...
            pre-defined roles: read|read-write|read-write-udf|sys-admin|user-admin
        DROP USER <user>
        SET PASSWORD <password> [FOR <user>]
        GRANT ROLE[S] <role1>,<role2>... TO <user>
        REVOKE ROLE[S] <role1>,<role2>... FROM <user>
        CREATE ROLE <role> PRIVILEGE[S] <priv1[.ns1[.set1]]>,<priv2[.ns2[.set2]]>...
            priv: read|read-write|read-write-udf|sys-admin|user-admin|data-admin
            ns:   namespace.  Applies to all namespaces if not set.
            set:  set name.  Applie to all sets within namespace if not set.
                  sys-admin, user-admin and data-admin can't be qualified with namespace or set.
        DROP ROLE <role>
        GRANT PRIVILEGE[S] <priv1[.ns1[.set1]]>,<priv2[.ns2[.set2]]>... TO <role>
        REVOKE PRIVILEGE[S] <priv1[.ns1[.set1]]>,<priv2[.ns2[.set2]]>... FROM <role>
        SHOW USER [<user>]
        SHOW USERS
        SHOW ROLE <role>
        SHOW ROLES

##### SETTINGS
        TIMEOUT                       (time in ms, default: 1000)
        RECORD_TTL                    (time in sec, default: 0)
        RECORD_PRINT_METADATA         (true | false, default false)
        VERBOSE                       (true | false, default false)
        ECHO                          (true | false, default false)
        FAIL_ON_CLUSTER_CHANGE        (true | false, default true, policy applies to scans)
        OUTPUT                        (TABLE | JSON, default TABLE)
        OUTPUT_TYPES                  (true | false, default true)
        LUA_USERPATH                  <path>, default : /opt/aerospike/usr/udf/lua
        LUA_SYSPATH                   <path>, default : /opt/aerospike/sys/udf/lua
        USE_SMD                       (true | false, default false)
        REPLICA_ANY                   (true | false, default false)
        DURABLE_DELETE                (true | false, default false)
        KEY_SEND                      (true | false, default false)

        To get the value of a setting, run:

            aql> GET <setting>

        To set the value of a setting, run:

            aql> SET <setting> <value>

##### OTHER
        RUN <filepath>
        HELP
        QUIT|EXIT|Q